# Importación Librerías

In [2]:
import pandas as pd
from io import StringIO
import gdown
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ast
from collections import Counter
from datetime import datetime
import seaborn as sns
import numpy as np
import reverse_geocoder as rg

# Importación Archivos

In [3]:
df_reviewsGoogle_CA=pd.read_parquet('../Archivos/ReviewsGoogle_CA.parquet')

In [ ]:
df_reviewsGoogle_CA.info()

Espacio que ocupa Reviews - Google: 228 Mb en parquet, 165 Mb en Dataframe

In [4]:
df_metadatos_CA=pd.read_parquet('../Archivos/metadatos_CA.parquet')

In [ ]:
df_metadatos_CA.info()

Espacio que ocupa Metadatos
Parquet: 85 Mb
Dataframe: 36,3 Mb

In [57]:
# Leer el archivo pickle en un DataFrame de pandas
df_businessYelp = pd.read_pickle('../Archivos/business.pkl')


In [ ]:
df_businessYelp.info()

Espacio que ocupa Business - Yelp: Archivo pickle 113,6 Mb, df 33,6 Mb



# Reviews Estados - Google

In [64]:
df_reviewsGoogle_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB


#### user_id

Es una entero de 21 cifras. No lo soporta en int de pandas, hay que pasar al int nativo de Python.
Para Pandas va a ser un object entonces, pero para Python un entero.
Sin nulos.

In [65]:
df_reviewsGoogle_CA['user_id']

0          1.089912e+20
1          1.112903e+20
2          1.126404e+20
3          1.174403e+20
4          1.005808e+20
               ...     
2699995    1.139124e+20
2699996    1.180461e+20
2699997    1.071072e+20
2699998    1.172404e+20
2699999    1.117537e+20
Name: user_id, Length: 2700000, dtype: float64

In [ ]:
df_reviewsGoogle_CA['user_id']=df_reviewsGoogle_CA['user_id'].apply(int)

In [66]:
df_reviewsGoogle_CA['user_id'].isna().sum()

0

#### name

Es un texto. El que más valor tiene, tiene 77 caracteres.
Hay caracteres en todos los idiomas (chino, japonés, etc).
Sin nulos.

In [67]:
cual=df_reviewsGoogle_CA['name'].apply(len)

In [68]:
cual.sort_values(ascending=False)

181901     77
864517     74
1574832    74
658053     74
1720881    74
           ..
1580582     1
1669924     1
1540558     1
2091968     1
133906      1
Name: name, Length: 2700000, dtype: int64

In [69]:
df_reviewsGoogle_CA['name'][181901]

'image system photographie Cruseilles Annecy Genève (Photographe Haute Savoie)'

In [70]:
df_reviewsGoogle_CA['name'].isna().sum()

0

In [71]:
df_reviewsGoogle_CA['name'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2700000 entries, 0 to 2699999
Series name: name
Non-Null Count    Dtype 
--------------    ----- 
2700000 non-null  object
dtypes: object(1)
memory usage: 20.6+ MB


#### time

Valores enteros de 13 cifras.
Marcas de tiempo en milisegundos desde la época Unix (1970-01-01).
Sin nulos.

In [72]:
df_reviewsGoogle_CA['time']

0          1609909927056
1          1612849648663
2          1583643882296
3          1551938216355
4          1494910901933
               ...      
2699995    1553382802382
2699996    1509453169393
2699997    1614453586811
2699998    1569739943903
2699999    1521661727317
Name: time, Length: 2700000, dtype: int64

In [73]:
df_reviewsGoogle_CA['time'].isna().sum()

0

In [74]:
df_reviewsGoogle_CA['time'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2700000 entries, 0 to 2699999
Series name: time
Non-Null Count    Dtype
--------------    -----
2700000 non-null  int64
dtypes: int64(1)
memory usage: 20.6 MB


#### rating

Enteros del 1 al 5.
Sin nulos.

In [75]:
df_reviewsGoogle_CA['rating'].unique()

array([5, 4, 1, 2, 3], dtype=int64)

In [76]:
df_reviewsGoogle_CA['rating'].isna().sum()

0

In [77]:
df_reviewsGoogle_CA['rating'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2700000 entries, 0 to 2699999
Series name: rating
Non-Null Count    Dtype
--------------    -----
2700000 non-null  int64
dtypes: int64(1)
memory usage: 20.6 MB


#### text

1170964 nulos (previo a eliminar los datos duplicados)

In [78]:
df_reviewsGoogle_CA[df_reviewsGoogle_CA['text']==' ']

,user_id,name,time,rating,text,pics,resp,gmap_id


In [79]:
cual = df_reviewsGoogle_CA['text'].fillna('').apply(len)

In [80]:
cual.sort_values(ascending= False).head(20)

739381     8360
1123967    6558
379508     5706
982612     5357
457539     4901
2440085    4847
83241      4558
869971     4513
1899203    4501
310166     4441
2176644    4423
187057     4362
1171707    4357
1780872    4353
1780879    4353
271700     4328
85666      4299
2622263    4298
2094237    4289
2119331    4266
Name: text, dtype: int64

In [81]:
cual=cual[cual != 0]

In [82]:
cual.describe()

count    1.529036e+06
mean     1.324193e+02
std      2.007083e+02
min      1.000000e+00
25%      3.200000e+01
50%      7.000000e+01
75%      1.540000e+02
max      8.360000e+03
Name: text, dtype: float64

In [83]:
df_reviewsGoogle_CA['text'].isna().sum()

1170964

In [84]:
df_reviewsGoogle_CA['text'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2700000 entries, 0 to 2699999
Series name: text
Non-Null Count    Dtype 
--------------    ----- 
1529036 non-null  object
dtypes: object(1)
memory usage: 20.6+ MB


#### pics

In [85]:
df_reviewsGoogle_CA['pics'][df_reviewsGoogle_CA['pics'].isna() == False]

24         [{'url': ['https://lh5.googleusercontent.com/p...
126        [{'url': ['https://lh5.googleusercontent.com/p...
128        [{'url': ['https://lh5.googleusercontent.com/p...
129        [{'url': ['https://lh5.googleusercontent.com/p...
130        [{'url': ['https://lh5.googleusercontent.com/p...
                                 ...                        
2699573    [{'url': ['https://lh5.googleusercontent.com/p...
2699656    [{'url': ['https://lh5.googleusercontent.com/p...
2699919    [{'url': ['https://lh5.googleusercontent.com/p...
2699939    [{'url': ['https://lh5.googleusercontent.com/p...
2699960    [{'url': ['https://lh5.googleusercontent.com/p...
Name: pics, Length: 99290, dtype: object

In [86]:
df_reviewsGoogle_CA['pics'][24]

array([{'url': array(['https://lh5.googleusercontent.com/p/AF1QipNBpj0NUQ3GblfewsZxQX_OrXs9QlW8R2yHB2eP=w150-h150-k-no-p'],
             dtype=object)}                                                                                                ],
      dtype=object)

In [87]:
df_reviewsGoogle_CA['pics'].isna().sum()

2600710

In [88]:
cual2 = df_reviewsGoogle_CA['pics'].fillna('').apply(lambda x: len(str(x)))

In [89]:
cual2.sort_values(ascending=False).unique()

array([7096, 7045, 6956, 6816, 6676, 6536, 6396, 6350, 6256, 6116, 5976,
       5933, 5836, 5696, 5556, 5416, 5276, 5136, 4996, 4856, 4716, 4576,
       4436, 4404, 4296, 4156, 4016, 3876, 3848, 3736, 3709, 3596, 3456,
       3431, 3316, 3176, 3153, 3036, 3014, 2896, 2875, 2756, 2736, 2616,
       2597, 2476, 2458, 2336, 2319, 2196, 2180, 2056, 2041, 1916, 1902,
       1776, 1763, 1636, 1624, 1496, 1485, 1356, 1346, 1336, 1216, 1207,
       1076, 1068, 1060,  936,  929,  922,  908,  796,  790,  784,  656,
        651,  516,  512,  508,  500,  376,  373,  370,  236,  234,  232,
          0], dtype=int64)

In [90]:
cual2[cual2 != 0].describe()

count    99290.000000
mean       518.283312
std        624.701521
min        232.000000
25%        236.000000
50%        376.000000
75%        516.000000
max       7096.000000
Name: pics, dtype: float64

#### resp

In [97]:
df_reviewsGoogle_CA['resp'][df_reviewsGoogle_CA['resp'].isna() == False][30]

{'text': 'Thanks so much for your business and for taking the time to post a great review. Glad to hear our crew was so helpful and friendly and that we were able to get the family outfitted quickly and efficiently. Look forward to hearing from you in the future.',
 'time': 1631072651706.0}

In [98]:
df_reviewsGoogle_CA['resp'].isna().sum()

2454831

In [99]:
cual2 = df_reviewsGoogle_CA['resp'].fillna('').apply(lambda x: len(str(x)))

In [100]:
cual2.sort_values(ascending=False).head(20)

132531     4043
90668      4042
237605     4019
692469     4003
1358752    3925
1033090    3875
259455     3821
1150136    3800
330673     3796
506628     3777
1083758    3770
657500     3767
532441     3726
1670422    3708
247223     3702
334364     3641
835899     3581
880217     3433
1537546    3423
291859     3322
Name: resp, dtype: int64

In [101]:
cual2[cual2 != 0].describe()

count    245169.000000
mean        176.725773
std         147.989795
min          38.000000
25%          85.000000
50%         138.000000
75%         225.000000
max        4043.000000
Name: resp, dtype: float64

#### gmap_id

In [103]:
df_reviewsGoogle_CA['gmap_id'].isna().sum()

0

In [104]:
cual2 = df_reviewsGoogle_CA['gmap_id'].apply(len)

In [105]:
cual2.unique()

array([37, 36, 35, 34, 22, 33], dtype=int64)

In [106]:
del cual
del cual2
del i

# Metadatos - Google

In [40]:
df_metadatos_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297293 entries, 0 to 297292
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              297293 non-null  object 
 1   address           297293 non-null  object 
 2   gmap_id           297293 non-null  object 
 3   description       31902 non-null   object 
 4   latitude          297293 non-null  float64
 5   longitude         297293 non-null  float64
 6   category          295387 non-null  object 
 7   avg_rating        297293 non-null  float64
 8   num_of_reviews    297293 non-null  int64  
 9   price             31580 non-null   object 
 10  hours             230103 non-null  object 
 11  MISC              244162 non-null  object 
 12  state             233398 non-null  object 
 13  relative_results  273256 non-null  object 
 14  url               297293 non-null  object 
 15  estado            297293 non-null  object 
dtypes: float64(3), int64

#### name

In [41]:
(df_metadatos_CA['name']==' ').sum()

0

In [42]:
df_metadatos_CA['name'].apply(len).sort_values(ascending=False).head(15)

66683     246
144499    245
154437    240
61097     230
117616    215
62992     215
88562     212
147578    212
226146    210
126532    209
242518    209
93873     208
118446    204
63941     201
238318    177
Name: name, dtype: int64

#### address

In [43]:
print((df_metadatos_CA['address']==' ').sum())
df_metadatos_CA['address'].apply(len).sort_values(ascending=False).head(5)
print(df_metadatos_CA['address'][df_metadatos_CA['address'].apply(len).idxmin()])

0
Weed, CA 96094


In [44]:
print(df_metadatos_CA['address'][df_metadatos_CA['address'].apply(len).idxmin()])

Weed, CA 96094


In [45]:
df_metadatos_CA['address'][294423]

'Cheviot Hills AC Repair & Installation, 1833 S La Cienega Blvd, Los Angeles, CA 90035'

In [48]:
comas = df_metadatos_CA['address'].apply(lambda x: str(x).count(','))

In [51]:
df_metadatos_CA['address'][comas.idxmax()]

'Imperial Jewelry & Loan Pawn Shop Norwalk CA (Loan, Sell & Buy Gold, Silver, Diamonds, Coins, Bullion, Jewelry, Watches, Guns, Hand Bags, Guitars & Musical Instruments, Electronics, Sports Equipment, Power Tools), 11405 Firestone Blvd E, Norwalk, CA 90650'

In [52]:
for i in range(20):
    print(str(i)+': '+str((comas == i).sum()))

0: 0
1: 70
2: 8565
3: 268220
4: 18591
5: 1470
6: 256
7: 66
8: 24
9: 15
10: 3
11: 5
12: 3
13: 1
14: 1
15: 2
16: 1
17: 0
18: 0
19: 0


In [107]:
del comas

#### gmap_id

In [ ]:
print((df_metadatos_CA['gmap_id']==' ').sum())
print(df_metadatos_CA['gmap_id'][df_metadatos_CA['gmap_id'].apply(len).idxmin()])
print(df_metadatos_CA['gmap_id'][df_metadatos_CA['gmap_id'].apply(len).idxmax()])
print(df_metadatos_CA['gmap_id'].apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['gmap_id'].apply(len).sort_values(ascending=False).tail(5))

0
0x0:0x13154eabf70b4b93
0x80c2c98c0e3c16fd:0x29ec8a728764fdf9
0         37
193271    37
193278    37
193277    37
193276    37
Name: gmap_id, dtype: int64
176870    34
3738      34
137970    33
130938    22
171772    22
Name: gmap_id, dtype: int64


#### description

In [ ]:
print((df_metadatos_CA['description']==' ').sum())
print(df_metadatos_CA['description'][df_metadatos_CA['description'].dropna().apply(len).idxmin()])
print(df_metadatos_CA['description'][df_metadatos_CA['description'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['description'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['description'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
Factory showroom selling
Old-school, colorful outpost serving homestyle Mexican dishes at the counter in a compact, no-frills interior.
225819    110
228380    110
213016    110
265524    109
71814     109
Name: description, dtype: int64
218734    50
153301    48
151928    43
195624    35
115459    24
Name: description, dtype: int64


In [ ]:
df_metadatos_CA['description'].dropna().apply(len).unique()

array([ 76,  98,  89,  95,  99,  80,  92,  86,  82,  85,  84,  91,  97,
       100,  68,  96,  90,  93,  60,  88,  94,  83,  70,  77,  87,  67,
        81,  61,  72,  79,  57,  75,  78,  69,  62,  74,  73,  66, 101,
        56, 103,  63,  65, 109,  50,  59,  71, 108,  24,  43,  48,  53,
        35,  64,  55, 110, 107, 105], dtype=int64)

#### latitude

In [ ]:
df_metadatos_CA['latitude'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 294593 entries, 0 to 294592
Series name: latitude
Non-Null Count   Dtype  
--------------   -----  
294593 non-null  float64
dtypes: float64(1)
memory usage: 2.2 MB


In [ ]:
df_metadatos_CA['latitude']

0         34.018891
1         34.058092
2         34.023669
3         34.036694
4         34.015505
            ...    
294588    33.611202
294589    33.713194
294590    33.163214
294591    33.644645
294592    33.643829
Name: latitude, Length: 294593, dtype: float64

#### longitude

In [ ]:
df_metadatos_CA['longitude']

0        -118.215290
1        -118.292130
2        -118.232930
3        -118.249421
4        -118.181839
             ...    
294588   -117.864362
294589   -117.932468
294590   -117.345829
294591   -117.919316
294592   -117.928050
Name: longitude, Length: 294593, dtype: float64

#### category

In [ ]:
print((df_metadatos_CA['category']==' ').sum())
print(df_metadatos_CA['category'][df_metadatos_CA['category'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['category'][df_metadatos_CA['category'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['category'][df_metadatos_CA['category'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['category'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['category'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
['Textile exporter']
<class 'list'>
['Non-profit organization', 'After school program', 'Child care agency', 'Computer training school', 'Convention center', 'Economic development agency', 'Family service center', 'Food bank', 'HIV testing center', 'Recreation center', 'Senior citizen center', 'STD testing service', 'Volunteer organization', 'Youth center']
101797    14
210979    13
222674    13
164787    12
223596    12
Name: category, dtype: int64
119851    1
119852    1
119856    1
119858    1
294592    1
Name: category, dtype: int64


In [ ]:
print(type(df_metadatos_CA['category'][df_metadatos_CA['category'].dropna().apply(len).idxmax()]))

<class 'list'>


In [ ]:
print(df_metadatos_CA['category'].dropna().apply(lambda lst: sum(len(str(item)) for item in lst)).sort_values(ascending=False).head(5))

248837    272
81090     267
77440     267
101797    267
33366     263
Name: category, dtype: int64


In [ ]:
df_metadatos_CA['category'].dropna().apply(lambda lst: sum(len(str(item)) for item in lst)).idxmax()

248837

In [ ]:
df_metadatos_CA['category'][248837]

['Musical instrument store',
 'Audio visual consultant',
 'Audio visual equipment rental service',
 'Audio visual equipment repair service',
 'Audio visual equipment supplier',
 'Music instructor',
 'Music school',
 'Musical instrument rental service',
 'Musical instrument repair shop',
 'Used musical instrument store']

In [ ]:
# Eliminar valores nulos y aplanar las listas en una sola lista de categorías
all_categories = df_metadatos_CA['category'].dropna().explode()

# Contar la frecuencia de cada categoría
category_counts = Counter(all_categories)

# Convertir el resultado a un DataFrame para una visualización más clara
category_counts_df = pd.DataFrame(category_counts.items(), columns=['Category', 'Count'])

In [ ]:
del all_categories
del category_counts

In [ ]:
category_counts_df

,Category,Count
0,Textile exporter,2
1,Korean restaurant,319
2,Fabric store,276
3,Restaurant,11505
4,Ski rental service,53
...,...,...
3849,Takoyaki restaurant,1
3850,Co-ed school,1
3851,Match box manufacturer,1
3852,Japanese cheap sweets shop,2


In [ ]:
category_counts_df.sort_values(by='Count', ascending=False, inplace=True)

In [ ]:
len(category_counts_df[category_counts_df['Count']<100])

2855

#### avg_rating

In [ ]:
df_metadatos_CA['avg_rating']

0         4.5
1         4.4
2         3.3
3         4.3
4         3.5
         ... 
294588    3.8
294589    3.7
294590    2.1
294591    3.0
294592    2.6
Name: avg_rating, Length: 294593, dtype: float64

#### num_of_reviews

In [ ]:
df_metadatos_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294593 entries, 0 to 294592
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              294593 non-null  object 
 1   address           294593 non-null  object 
 2   gmap_id           294593 non-null  object 
 3   description       31729 non-null   object 
 4   latitude          294593 non-null  float64
 5   longitude         294593 non-null  float64
 6   category          292712 non-null  object 
 7   avg_rating        294593 non-null  float64
 8   num_of_reviews    294593 non-null  int64  
 9   price             31438 non-null   object 
 10  hours             228087 non-null  object 
 11  MISC              242093 non-null  object 
 12  state             231355 non-null  object 
 13  relative_results  270853 non-null  object 
 14  url               294593 non-null  object 
 15  estado            294593 non-null  object 
dtypes: float64(3), int64

In [ ]:
df_metadatos_CA['num_of_reviews'].unique().min()

1

#### price

In [ ]:
df_metadatos_CA['price'].dropna().unique()

array(['$', '$$', '$$$', '$$$$', '₩', '₩₩', '₩₩₩', '₩₩₩₩'], dtype=object)

#### hours

In [ ]:
df_metadatos_CA['hours'][200803]

[['Monday', '8AM–6PM'],
 ['Tuesday', '8AM–6PM'],
 ['Wednesday', '8AM–6PM'],
 ['Thursday', '8AM–6PM'],
 ['Friday', '8AM–6PM'],
 ['Saturday', '8AM–6PM'],
 ['Sunday', '8AM–5PM']]

In [ ]:
print((df_metadatos_CA['hours']==' ').sum())
print(df_metadatos_CA['hours'][df_metadatos_CA['hours'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['hours'][df_metadatos_CA['hours'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['hours'][df_metadatos_CA['hours'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['hours'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['hours'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
[['Thursday', '6:30AM–6PM'], ['Friday', '6:30AM–6PM'], ['Saturday', '6:30AM–6PM'], ['Sunday', '7AM–12PM'], ['Monday', 'Closed'], ['Tuesday', '6:30AM–6PM'], ['Wednesday', '6:30AM–6PM']]
<class 'list'>
[['Thursday', '6:30AM–6PM'], ['Friday', '6:30AM–6PM'], ['Saturday', '6:30AM–6PM'], ['Sunday', '7AM–12PM'], ['Monday', 'Closed'], ['Tuesday', '6:30AM–6PM'], ['Wednesday', '6:30AM–6PM']]
1         7
200820    7
200803    7
200804    7
200806    7
Name: hours, dtype: int64
102875    7
102876    7
102878    7
102879    7
294592    7
Name: hours, dtype: int64


In [ ]:
print(df_metadatos_CA['hours'].dropna().apply(lambda lst: sum(len(str(item)) for item in lst)).sort_values(ascending=False).head(5))

104826    211
73985     211
74030     211
16492     211
20595     211
Name: hours, dtype: int64


In [ ]:
df_metadatos_CA['hours'].dropna().apply(lambda lst: sum(len(str(item)) for item in lst)).id

In [ ]:
df_metadatos_CA['hours'][104826]

[['Saturday', '12:01AM–11:59PM'],
 ['Sunday', '12:01AM–11:59PM'],
 ['Monday', '12:01AM–11:59PM'],
 ['Tuesday', '12:01AM–11:59PM'],
 ['Wednesday', '12:01AM–11:59PM'],
 ['Thursday', '12:01AM–11:59PM'],
 ['Friday', '12:01AM–11:59PM']]

#### MISC

In [ ]:
print((df_metadatos_CA['MISC']==' ').sum())
print(df_metadatos_CA['MISC'][df_metadatos_CA['MISC'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['MISC'][df_metadatos_CA['MISC'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['MISC'][df_metadatos_CA['MISC'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['MISC'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['MISC'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
{'Service options': ['In-store pickup']}
<class 'dict'>
{'From the business': ['Identifies as women-led'], 'Service options': ['Curbside pickup', 'No-contact delivery', 'Delivery', 'Takeout', 'Dine-in'], 'Health & safety': ['Mask required', 'Staff wear masks', 'Staff get temperature checks', 'Staff required to disinfect surfaces between visits'], 'Highlights': ['LGBTQ friendly', 'Transgender safespace'], 'Popular for': ['Lunch', 'Dinner', 'Solo dining'], 'Accessibility': ['Wheelchair accessible entrance', 'Wheelchair accessible parking lot', 'Wheelchair accessible restroom', 'Wheelchair accessible seating'], 'Offerings': ['Comfort food', 'Late-night food', 'Quick bite', 'Small plates', 'Vegetarian options'], 'Dining options': ['Breakfast', 'Lunch', 'Dinner', 'Catering', 'Dessert', 'Seating'], 'Amenities': ['Bar onsite', 'Good for kids', 'High chairs', 'Restroom', 'Wi-Fi'], 'Atmosphere': ['Casual'], 'Crowd': ['Family-friendly', 'Groups'], 'Planning': ['Accepts reservations'], 'Payment

In [ ]:
type(df_metadatos_CA['MISC'][271589])

dict

In [ ]:
#ver el que tiene mas caracteres
df_metadatos_CA['MISC'].dropna().apply(lambda d: sum(len(str(item)) for sublist in d.values() for item in sublist)).idxmax()

202512

In [ ]:
print(df_metadatos_CA['MISC'][202512])

{'Service options': ['Outdoor seating', 'Curbside pickup', 'No-contact delivery', 'Delivery', 'Takeaway', 'Dine-in'], 'Health and safety': ['Mask required', 'Staff wear masks', 'Staff get temperature checks', 'Staff required to disinfect surfaces between visits'], 'Highlights': ['Fireplace', 'Great beer selection', 'Great cocktails', 'Great coffee', 'Great dessert', 'Great wine list', 'LGBTQ-friendly', 'Transgender safe space'], 'Popular for': ['Lunch', 'Dinner'], 'Accessibility': ['Wheelchair-accessible car park', 'Wheelchair-accessible entrance', 'Wheelchair-accessible seating', 'Wheelchair-accessible toilet'], 'Offerings': ['Alcohol', 'Beer', 'Cocktails', 'Coffee', 'Comfort food', 'Happy-hour drinks', 'Happy-hour food', "Kids' menu", 'Late-night food', 'Organic dishes', 'Small plates', 'Spirits', 'Wine'], 'Dining options': ['Lunch', 'Dinner', 'Catering', 'Dessert', 'Seating'], 'Amenities': ['Bar on site', 'Good for kids', 'High chairs', 'Toilets', 'Wi-Fi'], 'Atmosphere': ['Cosy', 'H

In [122]:
df_metadatos_CA['MISC'][1]

{'Accessibility': array(['Wheelchair accessible entrance'], dtype=object),
 'Activities': None,
 'Amenities': array(['Good for kids'], dtype=object),
 'Atmosphere': array(['Casual'], dtype=object),
 'Crowd': None,
 'Dining options': None,
 'From the business': None,
 'Getting here': None,
 'Health & safety': None,
 'Health and safety': None,
 'Highlights': None,
 'Offerings': array(['Comfort food'], dtype=object),
 'Payments': None,
 'Planning': None,
 'Popular for': None,
 'Recycling': None,
 'Service options': array(['Takeout', 'Dine-in', 'Delivery'], dtype=object)}

##### Detalle MISC

In [6]:
def expand_dict_to_columns(row):
    if pd.isnull(row):
        return pd.Series()
    
    expanded_dict = {}
    for key, value in row.items():
        if isinstance(value, np.ndarray):
            expanded_dict[key] = list(value)
        else:
            expanded_dict[key] = value
    
    return pd.Series(expanded_dict)

# Aplicar la función a la columna 'MISC'
df_misc_expanded = df_metadatos_CA['MISC'].apply(expand_dict_to_columns)

# Combinar con el DataFrame original
df_metadatos_CA = pd.concat([df_metadatos_CA, df_misc_expanded], axis=1)

# Eliminar la columna original 'MISC' si ya no es necesaria
df_metadatos_CA.drop(columns=['MISC'], inplace=True)

del df_misc_expanded

In [12]:
columnas=['Accessibility', 'Activities',
       'Amenities', 'Atmosphere', 'Crowd', 'Dining options',
       'From the business', 'Getting here', 'Health & safety',
       'Health and safety', 'Highlights', 'Offerings', 'Payments', 'Planning',
       'Popular for', 'Recycling', 'Service options']

for i in columnas:
    print('Datos de la columna :'+str(i))
    print(df_metadatos_CA[i].value_counts().head(5))

Datos de la columna :Accessibility
Accessibility
[Wheelchair accessible entrance]                                                                                                      183771
[Wheelchair accessible entrance, Wheelchair accessible parking lot]                                                                    11793
[Wheelchair accessible entrance, Wheelchair accessible restroom]                                                                        5322
[Wheelchair accessible entrance, Wheelchair accessible parking lot, Wheelchair accessible restroom]                                     2363
[Wheelchair accessible entrance, Wheelchair accessible parking lot, Wheelchair accessible restroom, Wheelchair accessible seating]      2037
Name: count, dtype: int64
Datos de la columna :Activities
Activities
[Bicycle rental]    72
[Bicycle hire]       2
Name: count, dtype: int64
Datos de la columna :Amenities
Amenities
[Good for kids]                 21985
[Mechanic]                

In [9]:
df_metadatos_CA.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'state',
       'relative_results', 'url', 'estado', 'Accessibility', 'Activities',
       'Amenities', 'Atmosphere', 'Crowd', 'Dining options',
       'From the business', 'Getting here', 'Health & safety',
       'Health and safety', 'Highlights', 'Offerings', 'Payments', 'Planning',
       'Popular for', 'Recycling', 'Service options'],
      dtype='object')

#### state

In [ ]:
print((df_metadatos_CA['state']==' ').sum())
print(df_metadatos_CA['state'][df_metadatos_CA['state'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['state'][df_metadatos_CA['state'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['state'][df_metadatos_CA['state'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['state'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['state'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
Closed
<class 'str'>
Closes soon ⋅ 10:30PM ⋅ Opens 11:30AM Thu
64359     41
231564    41
275739    41
254600    40
288754    40
Name: state, dtype: int64
184277    6
254864    6
248782    6
145686    6
174266    6
Name: state, dtype: int64


#### relative_results 

In [ ]:
print((df_metadatos_CA['relative_results']==' ').sum())
print(df_metadatos_CA['relative_results'][df_metadatos_CA['relative_results'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['relative_results'][df_metadatos_CA['relative_results'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['relative_results'][df_metadatos_CA['relative_results'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['relative_results'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['relative_results'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
['0x808f7d903888260d:0x846a0c7caf8e80e2']
<class 'list'>
['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x80c2c79998f75fff:0xd7ca5c67e96fb778', '0x80c2b899146d7507:0xf4162c12c9cf65f8', '0x80c2c77f2d419951:0x26285631b21e324c', '0x80c2b8add9016015:0x15836f81a963b35f']
101816    5
130836    5
130838    5
130839    5
130840    5
Name: relative_results, dtype: int64
230235    1
138010    1
96067     1
187418    1
19353     1
Name: relative_results, dtype: int64


#### url

In [ ]:
print((df_metadatos_CA['url']==' ').sum())
print(df_metadatos_CA['url'][df_metadatos_CA['url'].dropna().apply(len).idxmin()])
print(type(df_metadatos_CA['url'][df_metadatos_CA['url'].dropna().apply(len).idxmax()]))
print(df_metadatos_CA['url'][df_metadatos_CA['url'].dropna().apply(len).idxmax()])
print(df_metadatos_CA['url'].dropna().apply(len).sort_values(ascending=False).head(5))
print(df_metadatos_CA['url'].dropna().apply(len).sort_values(ascending=False).tail(5))

0
https://www.google.com/maps/place//data=!4m2!3m1!1s0x0:0x13154eabf70b4b93?authuser=-1&hl=en&gl=us
<class 'str'>
https://www.google.com/maps/place/Point+Reyes+National+Seashore,+Point+Reyes+Station,+CA+94956/data=!4m2!3m1!1s0x8085da4972d11a1d:0x3568b993a474528f?authuser=-1&hl=en&gl=us
277970    172
186611    167
202402    167
15248     161
3241      159
Name: url, dtype: int64
192036    107
194950    107
194056    107
130938     97
171772     97
Name: url, dtype: int64


# Business - YELP

In [11]:
df_businessYelp.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [13]:
df_businessYelp.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Primero se observa una duplicación en las columnas. El segundo set de columnas están duplicadas, con lo que deben eliminarse las mismas.

In [4]:
# Eliminación de columnas
df_businessYelp.columns=['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id2', 'name2', 'address2',
       'city2', 'state2', 'postal_code2', 'latitude2', 'longitude2', 'stars2',
       'review_count2', 'is_open2', 'attributes2', 'categories2', 'hours2']

df_businessYelp.drop(columns=['business_id2', 'name2', 'address2',
       'city2', 'state2', 'postal_code2', 'latitude2', 'longitude2', 'stars2',
       'review_count2', 'is_open2', 'attributes2', 'categories2', 'hours2'], inplace=True)


### Ubicaciones

Se observan en el dataframe, a simple vista, valores de estado nulo, o ciudades con estados difrentes. Se procede a determinar la veracidad de esos datos.
Los Angeles tiene los códigos postales del 90001 al 91609. Y California del 90001 al 96162

In [7]:
#vemos que no existen valores nulos del código postal.
df_businessYelp['postal_code'].isna().sum()

0

In [17]:
# Crear la columna 'LA' basada en 'postal_code'
df_businessYelp['LA'] = df_businessYelp['postal_code'].apply(lambda x: 1 if x.isdigit() and 90001 <= int(x) <= 91609 else 0)
print(df_businessYelp['LA'].sum())

7


In [15]:
# Crear la columna 'LA' basada en 'postal_code'
df_businessYelp['CA'] = df_businessYelp['postal_code'].apply(lambda x: 1 if x.isdigit() and 90001 <= int(x) <= 96162 else 0)
df_businessYelp[df_businessYelp['CA']==1]['city'].value_counts()

city
Santa Barbara                       3827
Goleta                               798
Carpinteria                          298
Isla Vista                            94
Montecito                             93
Summerland                            41
Truckee                               11
Santa Barbara                          5
Santa Barbra                           2
Reno                                   2
Santa  Barbara                         1
Real Goleta                            1
Meridian                               1
Boise                                  1
SANTA BARBARA                          1
Sparks                                 1
Salinas                                1
Santa Barbara & Ventura Counties       1
South Lake Tahoe                       1
SANTA BARBARA AP                       1
Carpinteria                            1
Santa Barbara,                         1
Costa Mesa                             1
Cerritos                               1
Valencia   

In [21]:
#total de negocios en california
len(df_businessYelp[df_businessYelp['CA']==1])

5201

La mayoría de los negocios (el 94%) se encuentran entre Santa Barbara, Goleta y Carptinteria.

In [22]:
#hay solo 7 valores que figuran con código postal de los Angeles
df_businessYelp[df_businessYelp['LA']==1]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,LA,CA
12258,bw90rjFL02A_8rVE34m5Lg,Cosmetic Solutions,1010 Lula Lula Dr,Lula Lula,TN,90261,38.816486,-90.32175,3.5,17,1,"{'BusinessAcceptsCreditCards': 'False', 'Accep...","Cosmetic Surgeons, Pizza, Doctors, Beauty & Sp...",None,1,1
25883,BZAVgg5ybeX_a4w-FW0mbA,1-800-GOT-JUNK? Central Coast,"6520 Platt Ave, Ste 650",West Hill,DE,91307,34.525564,-119.81175,5.0,25,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Junk Removal & Hauling, Local Services","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",1,1
40532,l-zztqecz2EO6sCYuyLuag,Hellz Bellz,111 W 7th Street PH16,Los Angeles,PA,90041,39.661034,-75.569746,5.0,5,0,None,"Fashion, Women's Clothing, Shopping",None,1,1
56262,zAhDliPU8xRGGpF17n_lvw,Sears Auto Center,100 Los Cerritos Mall,Cerritos,PA,90703,34.436382,-119.749707,3.0,130,0,{'BusinessAcceptsCreditCards': 'True'},"Automotive, Oil Change Stations, Auto Repair, ...","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",1,1
74169,2kZ4EpcQ6dl5vtsLZKM2bQ,Farmers Insurance - Maureen Martinez,"24510 Town Center Dr, Ste 230",Valencia,AB,91355,34.421845,-119.656752,3.5,8,0,None,"Financial Services, Insurance","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,1
115102,LllYcETCcistMoHZuhalXA,Joyful Noise Entertainment,,Boise,FL,91390,43.615019,-116.202314,5.0,6,1,{'BusinessAcceptsCreditCards': 'True'},"DJs, Event Planning & Services",None,1,1
119578,ZG3sntPpL5mEebogPDid_w,So Cal City Group,1100 Santa Barbara St,Santa Barbara,CA,91306,34.424793,-119.700945,4.5,16,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Roofing, Contractors, Home Services, Home & Ga...","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ...",1,1


In [26]:
#latitudes de las principales localidades. Parecen ser certeras.
df_businessYelp.loc[(df_businessYelp['CA'] == 1) & (df_businessYelp['city'].isin(['Santa Barbara', 'Goleta', 'Carpinteria']))]['latitude'].astype(float).describe()

count    4923.000000
mean       34.425772
std         0.015331
min        34.378225
25%        34.418476
50%        34.423594
75%        34.435452
max        34.597239
Name: latitude, dtype: float64

In [27]:
#longitudes de las principales localidades. Parecen ser certeras.
df_businessYelp.loc[(df_businessYelp['CA'] == 1) & (df_businessYelp['city'].isin(['Santa Barbara', 'Goleta', 'Carpinteria']))]['longitude'].astype(float).describe()

count    4923.000000
mean     -119.723604
std         0.078201
min      -119.967185
25%      -119.751821
50%      -119.707054
75%      -119.694833
max      -119.466286
Name: longitude, dtype: float64

Los valores arrojados de longitud y latitud de esas localidades parecen ser certeros, por lo que se entiende que los datos de localidades son correctos.

In [33]:
# Se evaúan latitudes y longitudes para ver si coinciden con la localidad de los Angeles
# Función para evaluar las condiciones y retornar 1 o 0
def is_in_LA(row):
    try:
        longitude = float(row['longitude'])
        latitude = float(row['latitude'])
        if (-118.5 <= longitude <= -117.96) and (33.7 <= latitude <= 34.13):
            return 1
        else:
            return 0
    except ValueError:
        return 0  # Si no se puede convertir a float, retornar 0

# Crear la columna 'LA_gps' usando apply y la función is_in_LA
df_businessYelp['LA_gps'] = df_businessYelp.apply(is_in_LA, axis=1)

In [34]:
df_businessYelp['LA_gps'].sum()

0

No existen negocios con ubicación georeferenciada en la ciudad de Los Angeles.